<a href="https://colab.research.google.com/github/saharshayegan/Twitter_LDA_text_analysis/blob/main/English_Twitter_LDA_TextAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Run this and reset the runtime. Then proceed.
!pip install --upgrade pandas==1.2
!pip install -U gensim

## Installing and importing

In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords

from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel, LsiModel
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from gensim import matutils

from collections import defaultdict
import pickle

/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]
/usr/local/lib/python3.7/dist-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Installing Stuff

In [ ]:
# Package is in detached-head mode. Clone from Github, commit latest changes and run setup.py
! git clone https://github.com/bmabey/pyLDAvis
%cd pyLDAvis
! python setup.py build
! python setup.py install
%cd ..

In [ ]:
! python3.7 -m pip install funcy
! pip install funcy
import sys
sys.path.insert(0,'/usr/local/lib/python3.7/dist-packages/funcy-1.17-py3.7.egg')
import funcy
# !pip install --upgrade pandas==1.2


import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import pyLDAvis.gensim_models as gensimvis

ModuleNotFoundError: ignored

Load Twitter batch. File has 6 tab delimited columns

## Base

In [ ]:
import pandas as pd

filename= "/content/drive/MyDrive/Colab Notebooks/thesis/diseases.txt"
with open(filename) as f:
    diseases = [line.rstrip() for line in f]
print(diseases)

path = r'/content/drive/MyDrive/Colab Notebooks/thesis/' # use your path

# disease = diseases[0]

## Text preparation/cleanup for analysis

In [ ]:
# I've run this before. don't run it again.
for disease in diseases:
  print(disease)
  df = pd.read_csv(path+disease+'_tweets.csv', index_col=None, header=0)
  tweet_list_raw = df['Text'].tolist()
  # Text cleaning
  # Remove http
  link_pttrn = r"http\S+"
  tweet_list = [re.sub(link_pttrn, "", tweet) for tweet in tweet_list_raw]

  # Remove non-letter sequences
  nl_pttrn = r"[^a-zA-Z']+"
  tweet_list = [re.sub(nl_pttrn, " ", tweet) for tweet in tweet_list]

  # Lower case
  tweet_list = [tweet.lower() for tweet in tweet_list]

  # Remove white space at beginning and end of document
  ws_pttrn = r"(^ | $)"
  tweet_list = [re.sub(ws_pttrn, "", tweet) for tweet in tweet_list]
  # tweet_list[10:20]
  # Pre-processing
  # print(tweet_list_raw[501])
  # Post-processing
  # print(tweet_list[501])


  import nltk
  # STOPWORDS
  nltk_stop_words = set(stopwords.words("english"))
  def tokenize(tweet):
      return [token for token in simple_preprocess(tweet) if token not in nltk_stop_words]
  tweets = [tokenize(tweet) for tweet in tweet_list]
  tweets
  tweets[501]
  frequency = defaultdict(int)
  for tweet in tweets:
      for token in tweet:
          frequency[token] += 1
  freq_series = pd.Series(frequency)
  # freq_series.sort_values(ascending=False)[0:10]
  days = ['mon','tue','wed','thu','fri','sat','sun']
  months = ['jan', 'feb', 'mar','apr','may','jun',\
          'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

  irrel = ["rt", "amp"]
  uncommon_words = [w[0] for w in frequency.items() if w[1]<5]
  two_letter = [w[0] for w in frequency.items() if len(w[0])< 3]

  removed = uncommon_words + two_letter +days + months + irrel  
  print('Text is cleaned')

  # Process does take about 10-15 minutes, so the output is saved and loaded
  tweets_clean = [[token for token in tweet if token not in removed] for tweet in tweets]
  tweets_clean[501]
  pickle_out = open(path+disease+"_clean_final.pickle", "wb")
  pickle.dump(tweets_clean, pickle_out)
  pickle_out.close()

## LDA and Visualization

### The LDA and Visualize methods

In [ ]:
def run_lda(corpus, dictionary, clusters,path,disease):
  # Runtime about 5 mins
  print('Trying to run the models')
  tweet_topics = LdaModel(corpus = corpus,
                        id2word = dictionary,
                        num_topics = clusters,
                        passes= 10)
  pickle_out2 = open(path+disease+"_ffinalmodel_{}clusters.pickle".format(clusters), "wb")
  pickle.dump(tweet_topics, pickle_out2)
  pickle_out2.close()
  return tweet_topics

In [ ]:
def visualize(tweet_topics, corpus, dictionary, clusters, path, disease):
  ### Visualizing
  vis_data = gensimvis.prepare(tweet_topics, corpus, dictionary)
  # print(vis_data)
  file = open(path+disease+'_twitter-LDA_{}clusters.txt'.format(clusters), 'w')
  file.write(str(vis_data))
  file.close()
  import pyLDAvis
  pyLDAvis.display(vis_data)
  pyLDAvis.save_html(vis_data, path+disease+'_twitter-LDA_{}clusters.html'.format(clusters))

### For each topic:

In [ ]:
for disease in diseases:
  import pickle
  tweets_clean = pickle.load(open(path+disease+"_clean_final.pickle", "rb" ) )
  tweets_clean[501]
  dictionary = corpora.Dictionary(tweets_clean)
  corpus = [dictionary.doc2bow(tweet) for tweet in tweets_clean]


  #Run batch LDA and cross fingers
  clusters = 4

  run_lda(corpus, dictionary, clusters,path,disease)

  tweet_topics = pickle.load(open(  path+disease+"_ffinalmodel_{}clusters.pickle".format(clusters), "rb" ) )
  for i, topic in enumerate(tweet_topics.print_topics(15)):
      print (i, topic)

  ### INSTALLING STUFF WAS HERE

  ### Visualizing
  visualize(tweet_topics, corpus, dictionary, clusters, path, disease)


0 (0, '0.073*"copd" + 0.038*"disease" + 0.030*"chronic" + 0.029*"pulmonary" + 0.022*"patients" + 0.021*"obstructive" + 0.016*"risk" + 0.010*"study" + 0.010*"lung" + 0.010*"smoking"')
1 (1, '0.059*"copd" + 0.012*"asthma" + 0.010*"register" + 0.010*"march" + 0.009*"conference" + 0.008*"join" + 0.007*"available" + 0.006*"nursing" + 0.006*"respiratory" + 0.006*"research"')
2 (2, '0.084*"copd" + 0.026*"asthma" + 0.013*"lung" + 0.010*"help" + 0.010*"respiratory" + 0.009*"health" + 0.009*"patients" + 0.009*"treatment" + 0.008*"new" + 0.008*"people"')
3 (3, '0.066*"copd" + 0.027*"covid" + 0.011*"get" + 0.009*"people" + 0.007*"like" + 0.007*"cancer" + 0.007*"know" + 0.006*"got" + 0.006*"hospital" + 0.006*"heart"')


NameError: ignored

In [ ]:
!pip install -U gensim

In [ ]:
import pickle
disease = 'COPD'
clusters = 3

tweets_clean = pickle.load(open(path+disease+"_clean_final.pickle", "rb" ) )
dictionary = corpora.Dictionary(tweets_clean)
corpus = [dictionary.doc2bow(tweet) for tweet in tweets_clean]

lda_model = pickle.load(open(  path+disease+"_ffinalmodel_{}clusters.pickle".format(clusters), "rb" ) )
perplexity = lda_model.log_perplexity(corpus)

In [ ]:
print(perplexity)

-7.070407183057969
-7.105437433571481


### For all topics together

In [ ]:
import pandas as pd
import pickle

path = r'/content/drive/MyDrive/Colab Notebooks/thesis/' # use your path

all_tweets = []

for disease in diseases:
  tweets = pickle.load(open(path+disease+"_clean_final.pickle", "rb" ) )
  all_tweets+= tweets

dictionary = corpora.Dictionary(all_tweets)
corpus = [dictionary.doc2bow(tweet) for tweet in all_tweets]
clusters = 6

lda_model = run_lda(corpus, dictionary, clusters,path,'all')

visualize(lda_model, corpus, dictionary, clusters, path, 'all')

# TFIDF


In [ ]:
for disease in diseases:
  import pickle
  tweets_clean = pickle.load(open(path+disease+"_clean_final.pickle", "rb" ) )
  tweets_clean[501]
  dictionary = corpora.Dictionary(tweets_clean)
  corpus = [dictionary.doc2bow(tweet) for tweet in tweets_clean]


## Determine time evolution of tweets

In [ ]:
# extract all document-topic distritbutions to dictionnary
# document_key = list(tweet_df.date)
document_topic = {}

In [ ]:
for doc_id in range(len(corpus)):
    docbok = corpus[doc_id]
    doc_topics = tweet_topics.get_document_topics(docbok, 0)
    tmp = []
    for topic_id, topic_prob in doc_topics:
        tmp.append(topic_prob)
    document_topic[document_key[doc_id]] = tmp

In [ ]:
df = pd.DataFrame.from_dict(document_topic, orient='index')

In [ ]:
df.columns = ["Nuclear", "Professor_Crime", "Thaler_Nobel", "Cancer_Research", "Unionization", "Evening_Program", "Zimmer_FreeSpeech"]

In [ ]:
df.head(5)

In [ ]:
df.to_csv("/Users/Orest/Desktop/Big Data and Text Analytics/FP/timed_tweets.csv")

Graphs are produced with ggplot2 package in R.